In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("PySparkShell") \
    .getOrCreate()

In [3]:
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [4]:
CoronaBeer = spark.read.csv('Corona.csv',inferSchema=True, header=True)
HeinekenBeer = spark.read.csv('Heineken.csv',inferSchema=True, header=True)
GuinnessBeer = spark.read.csv('Guinness.csv',inferSchema=True, header=True)
CarlsbergBeer = spark.read.csv('Carlsberg.csv',inferSchema=True, header=True)
Covid19 = spark.read.csv('covid19',inferSchema=True, header=True)

In [5]:
Covid19.toPandas().head()

,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,tests_units
0,ABW,Aruba,2020-03-13,2,2,0,0,18.733,18.733,0.0,0.0,NaN,NaN,NaN,NaN,None
1,ABW,Aruba,2020-03-20,4,2,0,0,37.465,18.733,0.0,0.0,NaN,NaN,NaN,NaN,None
2,ABW,Aruba,2020-03-24,12,8,0,0,112.395,74.930,0.0,0.0,NaN,NaN,NaN,NaN,None
3,ABW,Aruba,2020-03-25,17,5,0,0,159.227,46.831,0.0,0.0,NaN,NaN,NaN,NaN,None
4,ABW,Aruba,2020-03-26,19,2,0,0,177.959,18.733,0.0,0.0,NaN,NaN,NaN,NaN,None


In [6]:
CoronaBeer = CoronaBeer.withColumn("RelevantBeer", lit("Corona"))
CoronaBeer = CoronaBeer.withColumn("iso_code", lit("MEX"))
CarlsbergBeer = CarlsbergBeer.withColumn("RelevantBeer", lit("Carlsberg"))
CarlsbergBeer = CarlsbergBeer.withColumn("iso_code", lit("DNK"))
GuinnessBeer = GuinnessBeer.withColumn("RelevantBeer", lit("Guinness"))
GuinnessBeer = GuinnessBeer.withColumn("iso_code", lit("IRL"))
HeinekenBeer = HeinekenBeer.withColumn("RelevantBeer", lit("Heineken"))
HeinekenBeer = HeinekenBeer.withColumn("iso_code", lit("NLD"))

In [7]:
HeinekenBeer.toPandas().head()

,Date,Open,High,Low,Close,Adj Close,Volume,RelevantBeer,iso_code
0,2020-01-02,95.680000,95.980003,94.959999,95.820000,94.496239,382782,Heineken,NLD
1,2020-01-03,95.260002,96.139999,95.220001,96.000000,94.673752,327670,Heineken,NLD
2,2020-01-06,96.300003,97.660004,95.620003,97.519997,96.172745,666897,Heineken,NLD
3,2020-01-07,97.900002,98.820000,97.480003,97.480003,96.133308,536847,Heineken,NLD
4,2020-01-08,97.099998,98.480003,97.040001,98.040001,96.685570,579833,Heineken,NLD


In [8]:
Beer=CarlsbergBeer.union(GuinnessBeer)
Beer=Beer.union(CoronaBeer)
Beer=Beer.union(HeinekenBeer)

In [9]:
Beer.toPandas().head()
Beer.createOrReplaceTempView("Beer")
Covid19.createOrReplaceTempView("Covid19")

In [10]:

BigData=spark\
.sql(\
     "SELECT  Covid19.iso_code, Covid19.location, Covid19.date, Covid19.total_cases, Covid19.new_cases, Covid19.total_deaths, Covid19.new_deaths, Covid19.total_cases_per_million, Covid19.new_cases_per_million, Covid19.total_deaths_per_million, Covid19.new_deaths_per_million ,Beer.RelevantBeer, Beer.Close\
     FROM Covid19, Beer \
     WHERE Covid19.iso_code=Beer.iso_code AND  Covid19.date=Beer.Date")

In [11]:
BigData


DataFrame[iso_code: string, location: string, date: string, total_cases: int, new_cases: int, total_deaths: int, new_deaths: int, total_cases_per_million: double, new_cases_per_million: double, total_deaths_per_million: double, new_deaths_per_million: double, RelevantBeer: string, Close: double]

In [13]:
BigData.toPandas().to_csv('BigData.csv', index = False)